In [2]:
import numpy as np
from dataset import Dataset

def test_for_different_covs(cov_0, cov_1):

    μ_0 = np.array([0.75,0.25])
    #cov_0 = np.array([[0.01, 0.005], [0.005, 0.01]])

    μ_1 = np.array([0.25, 0.75])
    #cov_1 = np.array([[0.01, 0.005], [0.005, 0.01]])

    gamma_prior_cov = np.array([0.01]) #0.01*np.array([0.1])

    ν = cov_0[1,1]


    # Number of samples to generate
    n_samples = 1000

    # Generate samples alternately
    samples = np.zeros((n_samples, 2))
    for i in range(n_samples):
        if i % 2 == 0:
            samples[i] = np.random.multivariate_normal(μ_0, cov_0)
        else:
            samples[i] = np.random.multivariate_normal(μ_1, cov_1)

    #print(f"==>> samples: {samples}")


    ds = Dataset(samples, emb_dim=2, N=n_samples, K=2)

    for i in range(0,len(ds.z_vars)):
        ds.z_vars[i].probs = [1.0, 0.0] if i % 2 == 0 else [0.0, 1.0]

    assumed_dof = 5 #= d+3

    ds.means_vars[0].prior_cov = cov_0
    ds.means_vars[0].mean = μ_0
    ds.means_vars[0].cov = cov_0

    ds.means_vars[1].prior_cov = cov_1
    ds.means_vars[1].mean = μ_1
    ds.means_vars[1].cov = cov_1


    ds.gamma_vars[0].prior_cov =  np.array([gamma_prior_cov / ν]) # np.array([gamma_prior_cov])
    ds.gamma_vars[0].mean = np.array([cov_0[0, 1]]) / np.sqrt(ν)
    ds.gamma_vars[0].cov = np.array([gamma_prior_cov / ν])
    ds.gamma_vars[0].nu = cov_0[-1,-1]

    ds.gamma_vars[1].prior_cov = np.array([gamma_prior_cov / ν]) # np.array([gamma_prior_cov]) 
    ds.gamma_vars[1].mean = np.array([cov_1[0, 1]]) / np.sqrt(ν)
    ds.gamma_vars[1].cov = np.array([gamma_prior_cov / ν])
    ds.gamma_vars[1].nu = cov_1[-1,-1]

    ds.sigma_star_vars[0].prior_scale = np.array([cov_0[0,0] - ds.gamma_vars[0].mean ** 2]) * (assumed_dof - ds.d)  # np.array([cov_0[0,0] * (assumed_dof - ds.d)])
    ds.sigma_star_vars[0].dof = 5
    ds.sigma_star_vars[0].prior_dof = 5
    ds.sigma_star_vars[0].scale = np.array([cov_0[0,0] - ds.gamma_vars[0].mean ** 2]) * (assumed_dof - ds.d)
    ds.sigma_star_vars[0].nu = cov_1[-1,-1]           

    ds.sigma_star_vars[0].first_moment = ds.sigma_star_vars[0].first_mom()
    ds.sigma_star_vars[0].second_moment = ds.sigma_star_vars[0].second_mom()     


    ds.sigma_star_vars[1].prior_scale = np.array([cov_1[0,0] - ds.gamma_vars[1].mean ** 2]) * (assumed_dof - ds.d)  #np.array([[cov_1[0,0] * (assumed_dof - ds.d)]])
    ds.sigma_star_vars[1].dof = 5
    ds.sigma_star_vars[1].prior_dof = 5
    ds.sigma_star_vars[1].scale = np.array([cov_1[0,0] - ds.gamma_vars[1].mean ** 2]) * (assumed_dof - ds.d)
    ds.sigma_star_vars[1].nu = cov_1[-1,-1]

    ds.sigma_star_vars[1].first_moment = ds.sigma_star_vars[1].first_mom()
    ds.sigma_star_vars[1].second_moment = ds.sigma_star_vars[1].second_mom()

    full_sigma_inv_estimates = [np.linalg.inv(cov_mat) for cov_mat in [cov_0, cov_1]]



    for i, (r_var, z_var) in enumerate(zip(ds.r_vars, ds.z_vars)):
        C=0
        D=0
        norm_datapoint = ds.normed_embds[i]
        norm_datapoint = norm_datapoint.reshape(-1, 1)

        for k in range (0, len(ds.z_vars[i].probs)):
            data_group = k
            sigma = ds.sigma_star_vars[data_group]
            γ = ds.gamma_vars[data_group]
            μ = ds.means_vars[data_group]

            sigma_inv = full_sigma_inv_estimates[data_group]

            C += z_var.probs[k] * np.matmul(np.matmul(norm_datapoint.T, sigma_inv), norm_datapoint)
            D += z_var.probs[k] * np.matmul(np.matmul(norm_datapoint.T, sigma_inv), μ.mean)


        r_var.alpha = C / 2
        r_var.beta = D / C

        r_var.update_moments(norm_datapoint)

        # r_var.first_moment = np.linalg.norm(ds.embds[i])

        
        # initialise phi variables

    # ds.phi_var.conc[0] = n_samples // 2
    # ds.phi_var.conc[1] = n_samples // 2 


    ds.dataset_vi(max_iter=5, real_cov=cov_0)
    return ds

In [3]:
def test_gamma_stuff(cov_0, cov_1):
    ds_fitted = test_for_different_covs(cov_0, cov_1)

    print(f"==>> ds_fitted.gamma_vars[0].mean: {ds_fitted.gamma_vars[0].mean}")
    print(f"nu: {ds_fitted.gamma_vars[0].nu}")

    mean_reshaped = ds_fitted.gamma_vars[0].mean.reshape(-1)

    return mean_reshaped, ds_fitted.gamma_vars[0].nu

In [5]:
import matplotlib.pyplot as plt

def generate_random_covariance_matrix():
    A = np.random.uniform(-1, 1, (2, 2))
    return np.dot(A, A.transpose())

# Generate 10 random 2x2 covariance matrices for cov_0 and cov_1
cov_matrices = [generate_random_covariance_matrix() for _ in range(10)]

means = []
nus = []

# Test for different cov_0 and cov_1 values
for cov_0 in cov_matrices:
    for cov_1 in cov_matrices:
        mean, nu = test_gamma_stuff(cov_0, cov_1)
        means.append(mean)
        nus.append(nu)

# Flatten the lists
means = np.concatenate(means)
nus = np.concatenate(nus)

# Plot the results
plt.figure(figsize=(10, 6))
plt.scatter(nus, means, alpha=0.7)
plt.title('Mean vs Nu for Different Covariance Values')
plt.xlabel('Nu')
plt.ylabel('Mean')
plt.grid(True)
plt.show()

NameError: name 'cov_1' is not defined